On importe les settings et les classes robots, croix, mur et sol depuis les fichiers settings.py et sprites.py. (détaillées dans game.ipynb)

In [13]:
from settings import *
from sprites import *

On crée la classe Game, sans le moteur graphique.

La classe a comme attributs:
    win, si la partie est gagnée (si une solution a été trouvée)
    grid, le tableau de la grille, composé de Mur et de Sol
    cross, la croix
    robots, la liste des positions des robots
    active_robot, l'index du robot actif dans la liste robots
    possibles, la liste des actions possibles
    moves, la liste des instructions effectuées
    max_generation, la génération actuelle de l'arbre
    Tree, la base de l'arbre
    allChildren, la liste de tous les points du dernier niveau de l'arbre
    list_index, l'index pour la liste allChildren

On ajoute les méthodes:
    directionToCoords(direction) qui prends en input une direction (u, d, l, r pour up, down, left, right) et qui output les coordonées du mouvement (ex: (6, 8))
    coordsToLetter(coords) qui prend en input des coordonées numériques (ex: (6, 8)) et output les coordonnées alphanumériques (ex: e7).
    quit(path) qui prend en input le chemin optimal, et qui print la solution, ligne par ligne.

In [14]:
class Game():
    def __init__(self, cross_pos:tuple, robots:list[tuple]) -> None:
        self.win = False
        self.grid:list[list[Floor | Wall]] = self.create_grid()
        self.cross = Cross(cross_pos)
        self.robots:list[tuple] = []
        self.active_robot = -1
        for i, robot in enumerate(robots):
            self.robots.append(robot[0])
            if robot[1]:
                self.active_robot = i
        self.possibles = ['u', 'd', 'l', 'r', 's']
        self.moves = []
        self.max_generation = 0
        self.Tree = Node([self.robots, self.active_robot, self.cross, self.grid], ['Optimal Path:'], self, 0)
        self.allChildren:list[list[Node]] = [[self.Tree], []]
        self.list_index = 1
             
    def create_grid(self) -> list:
        grid:list[list] = []
        for y, row in enumerate(GRID):
            grid.append([])
            for x, column in enumerate(row):
                if column == 1:
                    grid[y].append(Wall((x, y)))
                else:
                    grid[y].append(Floor((x, y)))
        return grid
      
    def switchRobot(self) -> None:
        self.active_robot = (self.active_robot + 1) % len(self.robots)

    def directionToCoords(self, direction:str) -> str:
        pos = self.robots[self.active_robot]
        on_robot = False
        if direction == "u":
            while not self.grid[pos[1]][pos[0]].is_wall and not on_robot:
                pos = (pos[0], pos[1] - 1)
                for robot in self.robots:
                    if pos == robot:
                        on_robot = True
            return (pos[0], pos[1]+1)
        elif direction == "d":
            while not self.grid[pos[1]][pos[0]].is_wall and not on_robot:
                pos = (pos[0], pos[1] + 1)
                for robot in self.robots:
                    if pos == robot:
                        on_robot = True
            return (pos[0], pos[1]-1)
        elif direction == "l":
            while not self.grid[pos[1]][pos[0]].is_wall and not on_robot:
                pos = (pos[0] - 1, pos[1])
                for robot in self.robots:
                    if pos == robot:
                        on_robot = True
            return (pos[0]+1, pos[1])
        elif direction == "r":
            while not self.grid[pos[1]][pos[0]].is_wall and not on_robot:
                pos = (pos[0] + 1, pos[1])
                for robot in self.robots:
                    if pos == robot:
                        on_robot = True
            return (pos[0]-1, pos[1])
    
    def coordsToLetter(self, coords:tuple[int]) -> str:
        return f'{ALPHABET[coords[0]]}{coords[1]+1}'

    def quit(self, path:list[str]):
        optimal_path = []
        for action in path:
            if ',' in action: # is a coordinate
                _action = action.split(',')
                coords = (int(_action[0]), int(_action[1]))
                optimal_path.append(self.coordsToLetter(coords))
            else:
                optimal_path.append(action)
            print(optimal_path[-1])
        
        print(f'Optimal solution found in {len(path)-1} moves')
        
        self.win = True
                
    def updateTree(self) -> list:
        self.list_index = (self.list_index + 1) % 2 # switches between 0 & 1
        if self.allChildren[self.list_index] != [self.Tree]:
            self.allChildren[self.list_index] = []
        for parent in self.allChildren[(self.list_index+1)%2]:
            for child in parent.children:
                self.allChildren[self.list_index].append(child)
            parent.children = []
        for node in self.allChildren[self.list_index]:
            node.generateChildren()

On ajoute aussi la méthode updateTree() qui va créer un nouveau niveau de branches dans l'arbre des possibilités.

On crée l'arbre en créant la classe Node, qui est un point dans l'arbre.
On lui donne comme attributs les valeurs du jeu (position des robots, grille, jeu...), et la liste des possibilités suivantes.
On crée les méthodes:
    checkVictory() qui vérifie si le robot est arrivé à destination et qui indique au jeu la fin de la recherche.
    generateChildren() qui, pour toutes les possibilités, vérifie si elle est valable, et crée une branche de l'arbre avec cette possibilité là avec la méthode generateChild(action).
    update(action) qui modifie la position du robot en fonction de l'action effectuée.


In [15]:
class Node(Game):
    def __init__(self, state:list, path:list, main_game:Game, generation:int) -> None:
        self.state = state
        self.moves:list = path
        self.children:list = []
        self.main_game = main_game
        
        self.robots:list[Robot] = state[0]
        self.cross:Cross = state[2]

        self.grid:list[list[Floor | Wall]] = self.state[3]
        self.active_robot = self.state[1]
        self.generation = generation
        if self.main_game.max_generation < self.generation:
            self.main_game.max_generation = self.generation
        self.possibles = POSSIBLES
        
    def checkVictory(self) -> None:
        if self.active_robot == 0 and self.robots[self.active_robot] == self.cross.pos:
            self.win = True
            self.main_game.quit(self.moves)
       
    def generateChild(self, action) -> Game:
        state  = [[], self.active_robot, self.cross, self.grid]
        moves = []
        main_game = self.main_game
        gen = self.generation + 1
        
        for robot in self.robots:
            state[0].append(robot)
        
        for move in self.moves:
            moves.append(move)
        
        node = self.returnChild(state, moves, main_game, gen)
        node.update(action)
        
        return node
    
    def returnChild(self, state, moves, main_game, gen):
        return Node(state, moves, main_game, gen)
    
    def generateChildren(self):
        for action in self.possibles:
            direction_useful = True
            active_robot_pos = self.robots[self.active_robot]
            if action == 'u' and self.grid[active_robot_pos[1]-1][active_robot_pos[0]].is_wall:
                direction_useful = False
            if action == 'd' and self.grid[active_robot_pos[1]+1][active_robot_pos[0]].is_wall:
                direction_useful = False
            if action == 'l' and self.grid[active_robot_pos[1]][active_robot_pos[0]-1].is_wall:
                direction_useful = False
            if action == 'r' and self.grid[active_robot_pos[1]][active_robot_pos[0]+1].is_wall:
                direction_useful = False
                
            if action in 'udlr':
                coords = self.directionToCoords(action)
                action = f'{coords[0]},{coords[1]}'
                
            for index in range(2, min(len(self.moves)+1, 7), 2):
                if self.moves[-index] == action:
                    direction_useful = False
            
            if type(action) == tuple and action == self.robots[self.active_robot]:
                    direction_useful = False
            
            if action != self.moves[-1] and direction_useful:
                self.children.append(self.generateChild(action))
    
    def update(self, action:str) -> None:
        if action == "s": 
            action = "switch"
        if "switch" in action:
            self.switchRobot()
            if len(self.moves) <= self.generation:
                self.moves.append(action)
        else:
            coordinates:tuple = (int(action.split(',')[0]), int(action.split(',')[1]))
            self.robots[self.active_robot] = coordinates
            if len(self.moves) <= self.generation:
                self.moves.append(action)
        self.checkVictory()

On ajoute le module time pour chronométrer la recherche de la solution, et on lance la recherche.

In [16]:
from time import time

start_time = time()
loop_time = start_time
game = Game(CROSS_POS, ROBOT_POS)
for i in range(MAX_ITERATIONS):
    print(f'Current level: {game.max_generation+1}')
    print(f'Total elapsed Time: {time()-start_time}s')
    print(f'Elapsed time this level: {time()-loop_time}s')
    loop_time = time()
    game.updateTree()
    if game.win: 
        print(f'Found solution in {time()-start_time}s')
        break

Current level: 1
Total elapsed Time: 0.0s
Elapsed time this level: 0.0s
Current level: 2
Total elapsed Time: 0.0s
Elapsed time this level: 0.0s
Current level: 3
Total elapsed Time: 0.0s
Elapsed time this level: 0.0s
Current level: 4
Total elapsed Time: 0.0s
Elapsed time this level: 0.0s
Current level: 5
Total elapsed Time: 0.0019593238830566406s
Elapsed time this level: 0.0019593238830566406s
Current level: 6
Total elapsed Time: 0.0019593238830566406s
Elapsed time this level: 0.0s
Current level: 7
Total elapsed Time: 0.007884740829467773s
Elapsed time this level: 0.005925416946411133s
Current level: 8
Total elapsed Time: 0.02696514129638672s
Elapsed time this level: 0.019080400466918945s
Current level: 9
Total elapsed Time: 0.08247685432434082s
Elapsed time this level: 0.0555117130279541s
Current level: 10
Total elapsed Time: 0.20077061653137207s
Elapsed time this level: 0.11829376220703125s
Current level: 11
Total elapsed Time: 0.5613296031951904s
Elapsed time this level: 0.3605589866